In [ ]:
from math import radians, cos, sin, asin, sqrt
from pyproj import Proj, Transformer
import urllib.request
import pandas as pd
import requests
import json
import arcpy
import glob
import os

gdb_path = r"...\LiDAR2Contours.gdb"
fc = gdb_path + r"\\test_fc"
wdir = r"OK_LiDAR/LAZ_data"

def getBBox(shp):
    global bbox
    
    print("Calculating Bounding Box")
    
    bbox = {}
    
    # get x and y, max and min. Return dict 
    for row in arcpy.da.SearchCursor(shp, ['SHAPE@']):
        extent = row[0].extent
        xmin, xmax = extent.XMin, extent.XMax
        ymin, ymax = extent.YMin, extent.YMax
        bbox['sref'] = extent.spatialReference
        
    transform = Transformer.from_proj(2267, 4326, always_xy=True).transform
    ymin_, xmin_ = transform(ymin, xmin)
    ymax_, xmax_ = transform(ymax, xmax)
    
    bbox['minX'] = ymin_
    bbox['maxX'] = ymax_
    bbox['minY'] = xmin_
    bbox['maxY'] = xmax_
    
    return bbox
    
def haversine(lon1, lat1, lon2, lat2):
   
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # unit conversion
    ft = (6371 * c) * 3280.84 
    
    return ft

def boxArea(bbox):
    
    topL_x, topL_y = bbox['minX'], bbox['maxY']
    botL_x, botL_y = bbox['minX'], bbox['minY']
    botR_x, botR_y = bbox['maxX'], bbox['minY']
    
    height = haversine(topL_x, topL_y, botL_x, botL_y)
    width = haversine(botL_x, botL_y, botR_x, botR_y)
    
    area = height * width
    
    return area

def boxCover(bbox, json_path):
    
    print("Calculating Haversine Distances")
    print("Calculating LiDAR Coverage")
    
    queryArea = []

    with open(json_path) as lpc:
        data = json.load(lpc)
        
        for key, values in data.items():
            if key == 'items':
                for dataset in values:
                    dataBox = dataset['boundingBox']
                    
                    if dataBox['minY'] < bbox['minY']:
                        dataBox['minY'] = bbox['minY']
                    if dataBox['minX'] < bbox['minX']:
                        dataBox['minX'] = bbox['minX']
                    if dataBox['maxY'] > bbox['maxY']:
                        dataBox['maxY'] = bbox['maxY']
                    if dataBox['maxX'] > bbox['maxX']:
                        dataBox['maxX'] = bbox['maxX']
                        
                    queryArea.append(boxArea(dataBox))
    
    querySum = sum(queryArea)
    aoiSum = boxArea(bbox)
    percentCover = (querySum/aoiSum) * 100
    
    return percentCover
    
def queryProd(bbox, json_path):
    
    print("Querying USGS LiDAR Products")
    
    url = fr"https://viewer.nationalmap.gov/tnmaccess/api/products?datasets=Lidar+Point+Cloud+%28LPC%29&bbox={bbox['minX']}%2C{bbox['minY']}%2C{bbox['maxX']}%2C{bbox['maxY']}&dateType=lastUpdated&outputFormat=JSON&version=1&_csrf=c38e717e-c29c-4b94-b11e-6a056c498025"
    
    try:
        req = requests.get(url)
        with open(json_path, 'w') as fp:
            json.dump(req.json(), fp, indent=2)
    
    except:
        print(url)
        print("Invalid Request URL")

def urlList(json_path):
    
    url_list = []
    with open(json_path) as lpc:
            data = json.load(lpc)
            
            for key, values in data.items():
                if key == 'items':
                    
                    for dataset in values:
                        url_list.append(dataset['downloadLazURL'])
                        
    return url_list

def downloadUrls(url_list, wd):
    
    for url in url_list:
    
        filename = os.path.split(url)[1]
        file = wd + r"\\" + filename 
        urllib.request.urlretrieve(url, file)
    
def main():
    
    # make temporary json file for LAZ files
    json_ = r"lidar_req.json"
    # construct bounding box for mask
    b_box = getBBox(fc)
    # query download urls
    queryProd(b_box, json_)
    # calculate lidar coverage
    coverage = boxCover(b_box, json_)
    # get list of download urls
    urls = urlList(json_)
    # delete json file
    #os.remove(json_)
    
    # check if coverage is acceptable
    if coverage < 95:
        print("Warning: Coverage Under 100%")
        
    else:
        print("Coverage Over 100%")
        
        # temp file geodatabase
        arcpy.CreateFileGDB_management(r"C:\Users\03844\OK_LiDAR\LAZ_data", "temp.gdb")
        gdb = wdir + r"\temp.gdb"
        
        # download urls
        print("Downloading LAZ files from USGS")
        downloadUrls(urls, wdir)
        
        # list laz files
        lazFiles = glob.glob(wdir + r"\*.laz")
        
        count = 1
        for laz in lazFiles:
            
            # convert LAZ to LAS, remove LAZ
            las = laz[:-4] + "_new.las"
            arcpy.conversion.ConvertLas(laz, las, 'NO_COMPRESSION')
            os.remove(laz)
            
            # convert LAS to LASD
            lasd = las[:-4] + "_new.lasd"
            arcpy.MakeLasDatasetLayer_management(las, lasd, class_code=[2])
            os.remove(las)
            
            # check units
            sr = arcpy.Describe(lasd).SpatialReference
            unit = sr.linearUnitName
            if unit == '':
                unit = sr.angularUnitName
                
            # LASD to raster
            count = str(count)
            ras = gdb + fr"\raster{count}"
            ras_res = 3
            
            if unit == "Meter":
                ras_res = float(ras_res) * 0.3048
                z_factor = 3.28084
                arcpy.conversion.LasDatasetToRaster(lasd, ras, "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", ras_res, z_factor)
            elif unit == "Foot_US":
                arcpy.conversion.LasDatasetToRaster(lasd, ras, "ELEVATION", "BINNING AVERAGE LINEAR", "FLOAT", "CELLSIZE", ras_res, 1)
            os.remove(lasd)     
            
            count += int(count)
            
        print("Rasters Created")
                
main()